## Imports

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
from keras.models import Model

import pandas as pd
import boto3
from time import time

import os
import io
import tempfile
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load Model

In [ ]:
model = tf.keras.models.load_model('/home/drevital/cs_video_processor/models/suzuyo')

## General Settings

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 200, 200
BATCH_SIZE = 32
NUM_RAND_IMAGES = 10
batch_size = min(BATCH_SIZE, NUM_RAND_IMAGES*2)

## Utility functions for S3 Image Data Generator

In [ ]:
def get_image(imname):
    client = boto3.client('s3')
    bucket = 'obstacles-classification'
    key = imname
    outfile = io.BytesIO()
    client.download_fileobj(bucket, key, outfile)
    outfile.seek(0)
    im = plt.imread(outfile, format='jpg')
    return im

def preprocess_image(im):
    im = im.reshape(im.shape[0], im.shape[1], 1)
    arr = keras.preprocessing.image.smart_resize(im,
                                                 (IMG_HEIGHT, IMG_WIDTH),
                                                 interpolation='bilinear')
    arr /= 255.0
    return arr

def preprocess_image_1(img):
    image = img.resize((200, 200))
    array = img_to_array(img)
    return(array)

## Define S3 Image Generator

In [ ]:
def s3_image_generator(obstacle_image_names, no_obstacle_image_names, batch_size):
    s3 = boto3.resource('s3')
    batch_input = []
    batch_output = [1]*NUM_RAND_IMAGES + [0]*NUM_RAND_IMAGES
    num_images = NUM_RAND_IMAGES*2
    
    batch_paths = np.array(obstacle_image_names)
    np.random.shuffle(batch_paths)
    
    for input_path in tqdm(batch_paths[:NUM_RAND_IMAGES]):
        im = get_image(input_path)
        im = preprocess_image(im)
        batch_input += [im]

    batch_paths = np.array(no_obstacle_image_names)
    np.random.shuffle(batch_paths)

    for input_path in tqdm(batch_paths[:NUM_RAND_IMAGES]):
        im = get_image(input_path)
        im = preprocess_image(im)
        batch_input += [im]

    for i in range(0, num_images, batch_size):
        x = np.array(batch_input[i:i+batch_size])
        y = np.array(batch_output[i:i+batch_size])
        yield(x, y)

## Function to read dataset file names from s3

In [ ]:
def get_dataset_image_names(dataset):
    client = boto3.client('s3')
    bucket = 'obstacles-classification'
    image_names = []

    paginator = client.get_paginator('list_objects')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=dataset)

    for page in page_iterator:
        for image_name in page['Contents']:
            if image_name['Key'].split('.')[-1] == 'jpg':
                image_names.append(image_name['Key'])
                
    return image_names

## Function to display Confusion Matrix

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('No Obstacles Detected (True Negatives): ', cm[0][0])
  print('No Obstacles Incorrectly Detected (False Positives): ', cm[0][1])
  print('Obstacles Missed (False Negatives): ', cm[1][0])
  print('Obstacles Detected (True Positives): ', cm[1][1])
  print('Total Obstacles: ', np.sum(cm[1]))

## Fetch list of files in Evaluation Dataset to serve the S3 file-generator

In [ ]:
dataset = 'suzuyo/eval/obstacle'
obstacle_image_names = get_dataset_image_names(dataset)
dataset = 'suzuyo/eval/no_obstacle'
no_obstacle_image_names = get_dataset_image_names(dataset)
num_obstacles = len(obstacle_image_names)
num_no_obstacles = len(no_obstacle_image_names)
num_images = num_obstacles + num_no_obstacles

## Evaluate model and print metrics

In [ ]:
metrics = model.evaluate_generator(s3_image_generator(obstacle_image_names,
                                                      no_obstacle_image_names,
                                                      batch_size),
                                   verbose=1)

for name, value in zip(model.metrics_names, metrics):
  print(name, ': ', value)
print()

## Predict with the model and print prediction charts

In [ ]:
labels = np.array([1]*(NUM_RAND_IMAGES) + [0]*(NUM_RAND_IMAGES))
num_prediction_batches = math.ceil((NUM_RAND_IMAGES*2)/batch_size)

predictions = model.predict_generator(s3_image_generator(obstacle_image_names, 
                                                         no_obstacle_image_names, 
                                                         batch_size),
                                      num_prediction_batches,
                                      verbose=1)

plot_cm(labels, predictions) # Default: threshold = 0.5
plot_cm(labels, predictions, p=0.25)
plot_cm(labels, predictions, p=0.75)